In [ ]:
import pandas as pd
import recordlinkage
import numpy as np

# PATH al CSV di input
schema_file = "main_outputs/final_mediated_schema.csv"

# Caricamento dati e selezione colonne
df = pd.read_csv(schema_file, low_memory=False)

# Elenco di colonne di esempio su cui calcoleremo le similarità
# Adatta in base alle colonne effettive che vuoi comparare
columns_for_comparison = [
    'company_name',
    'industry',
    'headquarters_city',
    'headquarters_country',
    'ownership'
]

# Tieni solo queste colonne (se esistono nel CSV) più eventualmente altre
available_cols = [c for c in columns_for_comparison if c in df.columns]
df = df[available_cols].copy()

# Rimuovi eventuali righe prive di 'company_name'
if 'company_name' in df.columns:
    df.dropna(subset=['company_name'], inplace=True)
    
df.fillna('', inplace=True)  # Riempi i NaN con stringa vuota
df.reset_index(drop=True, inplace=True)

print(f"Numero di record dopo pulizia: {len(df)}")

#############################
# 1) Definiamo 2 indexer
#############################
indexer_industry = recordlinkage.Index()
indexer_industry.block('industry')  # blocca in base all'exact match di 'industry'

indexer_city = recordlinkage.Index()
indexer_city.block('headquarters_city')  # blocca in base all'exact match di 'headquarters_city'


#############################
# 2) Funzione di pipeline
#############################
def run_blocking_and_matching(df, indexer, strategy_name):
    """
    Esegue:
    - Creazione candidate pairs dal blocking
    - Calcolo similarità sulle colonne fornite
    - Filtro in base a soglie di similarità
    - Salvataggio CSV
    """
    print(f"\n=== Avvio pipeline per: {strategy_name} ===")

    # a) Creazione candidate pairs
    candidate_pairs = indexer.index(df)
    print(f" -> Candidati generati: {len(candidate_pairs)}")

    # b) Definizione comparatore
    compare = recordlinkage.Compare()

    # Esempio di soglie personalizzate su alcune colonne
    # Puoi modificare i parametri method='...', threshold=..., etc.
    compare.string('company_name', 'company_name', method='jarowinkler', label='name_sim')
    compare.string('industry', 'industry', method='jarowinkler', label='industry_sim')
    compare.string('headquarters_city', 'headquarters_city', method='jarowinkler', label='city_sim')
    
    if 'headquarters_country' in df.columns:
        compare.string('headquarters_country', 'headquarters_country', 
                       method='jarowinkler', label='country_sim')
    if 'ownership' in df.columns:
        compare.string('ownership', 'ownership', method='jarowinkler', label='ownership_sim')

    # c) Calcolo matrix similarità
    similarity_matrix = compare.compute(candidate_pairs, df)
    print(f" -> Dimensioni matrice similarità: {similarity_matrix.shape}")

    # d) Definiamo una strategia di filtro.
    #    Ad esempio, potremmo creare un punteggio pesato 
    #    (oppure soglie multiple).
    
    # Esempio: pesiamo di più il name_sim, poi industry_sim, city_sim, etc.
    # Se una colonna non esiste tra le features, usiamo 0 come fallback.
    similarity_matrix['weighted_score'] = (
        similarity_matrix.get('name_sim', 0)*0.4 +
        similarity_matrix.get('industry_sim', 0)*0.3 +
        similarity_matrix.get('city_sim', 0)*0.2 +
        similarity_matrix.get('country_sim', 0)*0.05 +
        similarity_matrix.get('ownership_sim', 0)*0.05
    )

    # Filtro: ad esempio consideriamo "match" se weighted_score > 0.75
    # E, in più, se vogliamo essere certi che name_sim superi una certa soglia
    # (es. 0.80), potremmo aggiungere:
    matches = similarity_matrix[
        (similarity_matrix['weighted_score'] > 0.75) &
        (similarity_matrix.get('name_sim', 0) > 0.80)
    ]

    print(f" -> Match trovati dopo filtro: {len(matches)}")

    # e) Recupero dei valori dal DF
    matches = matches.reset_index()
    matches.rename(columns={'level_0': 'id_left', 'level_1': 'id_right'}, inplace=True)

    # Join con il DF originale per vedere i record
    matches = matches.merge(df, left_on='id_left', right_index=True, how='left', suffixes=('', '_left'))
    matches = matches.merge(df, left_on='id_right', right_index=True, how='left', suffixes=('_left', '_right'))

    # (Facoltativo) Rimuoviamo righe in cui "company_name_left == company_name_right"
    # se vogliamo escludere duplicati perfetti di nome
    matches = matches[matches['company_name_left'] != matches['company_name_right']]

    # Salvataggio
    outfile = f"matched_companies_{strategy_name}.csv"
    matches.to_csv(outfile, index=False)
    print(f"✅ File '{outfile}' generato con {len(matches)} righe.\n")


#############################
# 3) Eseguiamo la pipeline separatamente
#############################
run_blocking_and_matching(df, indexer_industry, "block_by_industry")
run_blocking_and_matching(df, indexer_city, "block_by_city")

print("=== Processo completato! ===")



[ RUN ] Strategy=block_by_industry, Threshold=strict
   --> 0 matches salvati in 'matched_block_by_industry_strict.csv'

[ RUN ] Strategy=block_by_industry, Threshold=relaxed
   --> 0 matches salvati in 'matched_block_by_industry_relaxed.csv'

[ RUN ] Strategy=block_by_city, Threshold=strict
   --> 0 matches salvati in 'matched_block_by_city_strict.csv'

[ RUN ] Strategy=block_by_city, Threshold=relaxed
   --> 0 matches salvati in 'matched_block_by_city_relaxed.csv'

=== RISULTATI ===
  threshold_name  total_pairs  num_matches  avg_similarity  execution_time  \
0         strict       245246            0               0        0.995567   
1        relaxed       245246            0               0        0.982007   
2         strict       788361            0               0        4.307461   
3        relaxed       788361            0               0        4.182340   

  sample_matches      strategy_name     score  
0             []  block_by_industry  0.200444  
1             []  bloc